### xG Rolling Plots - TeamName

Import libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from highlight_text import fig_text

Data

In [ ]:
competition_2021 = pd.read_csv("../data/raw/Fbref-competition-2021.csv")
competition_2122 = pd.read_csv("../data/raw/Fbref-competition-2021.csv")

In [ ]:
competition_2021 = competition_2021[['Data', 'Casa', 'xG', 'xG.1', 'Ospiti']]
competition_2122 = competition_2122[['Data', 'Casa', 'xG', 'xG.1', 'Ospiti']]

In [ ]:
# column rename
competition_2021.rename(columns={"Data": "Date", "Casa": "HomeTeam", "xG": "xG HomeTeam", "xG.1": "xG AwayTeam", "Ospiti": "AwayTeam"}, inplace=True)
competition_2122.rename(columns={"Data": "Date", "Casa": "HomeTeam", "xG": "xG HomeTeam", "xG.1": "xG AwayTeam", "Ospiti": "AwayTeam"}, inplace=True)

In [ ]:
# column date as timestamp
competition_2021['Date'] = pd.to_datetime(competition_2021['Date'])
competition_2122['Date'] = pd.to_datetime(competition_2122['Date'])

In [ ]:
team_2021 = competition_2021.loc[(competition_2021.HomeTeam == 'TeamName') | (competition_2021.AwayTeam == 'TeamName')]
team_2122 = competition_2122.loc[(competition_2122.HomeTeam == 'TeamName') | (competition_2122.AwayTeam == 'TeamName')]

In [ ]:
team_2021.to_csv("../data/Fbref-team_2021.csv")
team_2122.to_csv("../data/Fbref-team_2122.csv")

#### Data Manipulation

append the two dataframe

In [ ]:
teamname = pd.concat([team_2021, team_2122])
len(teamname)

In [ ]:
teamname.to_csv("../data/teamname-xG.csv")

we need to create a series for both expected goals created and conceded regardless if the team played at home or away

create a new DataFrame with six columns: team, opponent, variable, value, venue and date

we'll split our df into two and then concatenate them back together

by adding the venue column we could even deepen our analysis to only consider home or away performance

In [ ]:
teamname.columns

In [ ]:
home_df = teamname.copy()

In [ ]:
home_df = home_df.melt(id_vars=['Date', 'HomeTeam', 'AwayTeam'])
home_df['Venue'] = "H"

In [ ]:
home_df.rename(columns= {'HomeTeam': 'Team', 'AwayTeam': 'Opponent'}, inplace=True)
home_df.replace({"variable":{'xG HomeTeam': 'xG_for', 'xG AwayTeam': 'xG_ag'}}, inplace=True)

same for away data

In [ ]:
away_df = teamname.copy()

In [ ]:
away_df = away_df.melt(id_vars=['Date', 'AwayTeam', 'HomeTeam'])
away_df['Venue'] = "A"

In [ ]:
away_df.rename(columns= {'AwayTeam': 'Team', 'HomeTeam': 'Opponent'}, inplace=True)
away_df.replace({"variable":{'xG AwayTeam': 'xG_for', 'xG HomeTeam': 'xG_ag'}}, inplace=True)

join it back together

In [ ]:
df = pd.concat([home_df, away_df]).reset_index(drop = True)

In [ ]:
df.head()

filter the records related to AC Milan and compute the rolling average for the expected goals data

In [ ]:
# filter team data
df = df[df['Team'] == "TeamName"].reset_index(drop=True)
df = df.sort_values(by='Date')

In [ ]:
# xG conceded and xG created
Y_for = df[df['variable'] == "xG_for"].reset_index(drop=True)
Y_ag = df[df['variable'] == "xG_ag"].reset_index(drop=True)
X = pd.Series(range(len(Y_for)))

In [ ]:
# compute the rolling average (min_periods is used for the partial average)
# here we're using a 10 game rolling average
Y_for = Y_for.rolling(window=10, min_periods=0).mean()
Y_ag = Y_ag.rolling(window=10, min_periods=0).mean()

#### Data Visualization

In [ ]:
from PIL import Image
import urllib

In [ ]:
fig = plt.figure(figsize=(4.5, 2.5), dpi=200, facecolor="#EFE9E6")
ax = plt.subplot(111, facecolor="#EFE9E6")

# remove top and right spines and change the color
ax.spines[['top', 'right']].set_visible(False)
ax.spines[['left', 'bottom']].set_color("grey")

# set the grid
ax.grid(
    visible = True, 
    lw = 0.75,
    ls = ":",
    color = "lightgrey"
)

line_1 = ax.plot(X, Y_for, color="#??????", zorder=4)
line_2 = ax.plot(X, Y_ag, color="#??????", zorder=4)

ax.set_ylim(0)
# add a line to mark the division between seasons
ax.plot(
    [38,38], # 38 games per season
    [ax.get_ylim()[0], ax.get_ylim()[1]],
    ls = ":",
    lw = 1.25,
    color = "grey",
    zorder = 2
)

# annotation with data coordinates and offset points
ax.annotate(
    xy = (38, .55),
    xytext = (20, 10),
    textcoords = 'offset points',
    text = 'Change in CompetitionName season',
    size = 6,
    color = "grey",
    arrowprops = dict(
        arrowstyle="->", shrinkA=0, shrinkB=5, color="grey", linewidth=0.75,
        connectionstyle="angle3,angleA=50,angleB=-30"
    ) # arrow to connect annotation
)

# fill between
ax.fill_between(
    X,
    Y_ag['value'],
    Y_for['value'],
    where = Y_for['value'] > Y_ag['value'],
    interpolate = True,
    alpha = 0.85,
    zorder = 3,
    color = line_1[0].get_color()
)

ax.fill_between(
    X,
    Y_ag['value'],
    Y_for['value'],
    where = Y_ag['value'] > Y_for['value'],
    interpolate = True,
    alpha = 0.85,
    color = line_2[0].get_color()
)

# customize the ticks to match spinecolor and adjust label size
ax.tick_params(
    color = "grey",
    length = 5,
    which = "major",
    labelsize = 6,
    labelcolor = "grey",
    zorder = 3
)

# set x-axis major tick positions to only 19 game multiples
ax.xaxis.set_major_locator(ticker.MultipleLocator(19))
# set y-axis major tick positions to only 0.5xG multiples
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.5))

# title and subtitle for the legend
fig_text(
    x = 0.12, y = 1.1,
    s = "TeamName",
    color = "black",
    weight = "bold",
    size = 10,
    family = "DM Sans",
    annotationbbox_kw={"xycoords": "figure fraction"}
)

fig_text(
    x = 0.12, y = 1.02,
    s = "Expected goals <created> and <conceded> | 10-match rolling average \nCompetitionName seasons 20/21 and 21/22",
    highlight_textprops=[
        {"color": line_1[0].get_color(), "weight": "bold"},
        {"color": line_2[0].get_color(), "weight": "bold"}
    ],
    color = "black",
    size = 6,
    annotationbbox_kw={"xycoords": "figure fraction"}
)

fotmob_url = "https://images.fotmob.com/image_resources/logo/teamlogo/"

logo_ax = fig.add_axes([0.75, .99, 0.13, 0.13], zorder=1)
club_icon = Image.open(urllib.request.urlopen(f"{fotmob_url}8564.png"))
logo_ax.imshow(club_icon)
logo_ax.axis("off")

save fig

In [ ]:
fig.savefig('../figures/TeamName-xG-rolling-plot.png', bbox_inches='tight')

team color palette

#??????
#??????